# EXAONE-4.0 Tool Calling 예제

EXAONE-4.0 모델의 tool calling 기능을 사용하는 실용적인 가이드입니다.

## Tool Calling이란?
언어 모델이 외부 함수나 API를 호출할 수 있게 해주는 기능으로, 실시간 데이터 접근, 계산, 시스템 제어 등을 수행할 수 있습니다.

## 1. 라이브러리 임포트

먼저 tool calling에 필요한 라이브러리들을 임포트합니다.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import re
import torch

print(f"✅ 라이브러리 로드 완료 | PyTorch: {torch.__version__} | CUDA: {torch.cuda.is_available()}")

/home/etri12/anaconda3/envs/korean_llm_/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


라이브러리 임포트 완료!
PyTorch 버전: 2.7.1+cu126
CUDA 사용 가능: True


## 2. 모델 로드

EXAONE-4.0-1.2B 모델과 토크나이저를 로드합니다. 이 모델은 tool calling을 지원합니다.

In [ ]:
model_name = "LGAI-EXAONE/EXAONE-4.0-1.2B"

# 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="bfloat16", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(f"✅ 모델 로드 완료: {model_name}")
print(f"✅ Tool calling 지원: {hasattr(tokenizer, 'chat_template')}")

# Chat template 일부 확인
if hasattr(tokenizer, 'chat_template'):
    template_preview = tokenizer.chat_template[:200] + "..." if len(tokenizer.chat_template) > 200 else tokenizer.chat_template
    print(f"Chat template 미리보기:\n{template_preview}")

모델과 토크나이저를 로드하는 중...
✅ 모델 로드 완료: LGAI-EXAONE/EXAONE-4.0-1.2B
✅ 토크나이저 tool calling 지원: True
Chat template 미리보기:
{%- if not skip_think is defined %}
  {%- set skip_think = true %}
{%- endif %}

{%- set role_indicators = {
    'user': '[|user|]\n',
    'assistant': '[|assistant|]\n',
    'system': '[|system|]\n',...


## 3. 툴 함수 정의

In [ ]:
def get_weather(location):
    """날씨 정보 조회"""
    weather_data = {
        "Seoul": "맑음, 22°C", "서울": "맑음, 22°C",
        "Tokyo": "흐림, 18°C", "도쿄": "흐림, 18°C", 
        "New York": "비, 15°C", "뉴욕": "비, 15°C"
    }
    return weather_data.get(location, f"{location}의 날씨 정보를 찾을 수 없습니다")

def calculate(operation, a, b):
    """계산기"""
    try:
        a, b = float(a), float(b)
        ops = {"add": a + b, "subtract": a - b, "multiply": a * b, "divide": a / b if b != 0 else None}
        result = ops.get(operation.replace("더하기", "add").replace("빼기", "subtract").replace("곱하기", "multiply").replace("나누기", "divide"))
        return f"{a} {operation} {b} = {result}" if result is not None else "오류: 0으로 나눌 수 없습니다"
    except:
        return "오류: 유효하지 않은 입력"

def get_current_time():
    """현재 시간"""
    from datetime import datetime
    return f"현재 시간: {datetime.now().strftime('%Y년 %m월 %d일 %H시 %M분')}"

# 간단한 테스트
print("✅ 툴 함수 정의 완료")
print(f"날씨: {get_weather('서울')}")
print(f"계산: {calculate('add', 10, 5)}")
print(f"시간: {get_current_time()}")

🌤️  날씨 함수 테스트:
  서울: 맑음, 22°C
  Tokyo: 흐림, 18°C

🧮 계산기 함수 테스트:
  더하기: 10.0 + 5.0 = 15.0
  곱하기: 7.0 × 8.0 = 56.0

⏰ 시간 함수 테스트:
  현재 시간: 2025년 07월 16일 16시 38분 41초


## 4. 툴 스키마 정의

모델이 사용할 수 있는 툴들의 스키마를 정의합니다. 각 툴은 이름, 설명, 파라미터 스키마를 포함합니다.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "특정 위치의 날씨 정보 조회",
            "parameters": {
                "type": "object",
                "properties": {"location": {"type": "string", "description": "위치 (예: 서울, Tokyo)"}},
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate", 
            "description": "기본 수학 계산",
            "parameters": {
                "type": "object",
                "properties": {
                    "operation": {"type": "string", "description": "연산: add, subtract, multiply, divide"},
                    "a": {"type": "number", "description": "첫 번째 숫자"},
                    "b": {"type": "number", "description": "두 번째 숫자"}
                },
                "required": ["operation", "a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "현재 시간 조회",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    }
]

print(f"✅ {len(tools)}개 툴 정의 완료: {[t['function']['name'] for t in tools]}")

✅ 총 3개의 툴이 정의되었습니다:
  1. get_weather: 특정 위치의 현재 날씨 정보를 조회합니다
  2. calculate: 기본적인 수학 계산을 수행합니다
  3. get_current_time: 현재 날짜와 시간을 반환합니다

📋 첫 번째 툴의 상세 구조:
{
  "type": "function",
  "function": {
    "name": "get_weather",
    "description": "특정 위치의 현재 날씨 정보를 조회합니다",
    "parameters": {
      "type": "object",
      "properties": {
        "location": {
          "type": "string",
          "description": "날씨를 조회할 위치 (예: 서울, Tokyo, New York)"
        }
      },
      "required": [
        "location"
      ]
    }
  }
}


## 5. 툴 실행 함수

모델이 요청한 툴 호출을 실제로 실행하는 함수를 구현합니다.

In [ ]:
def execute_tool_call(tool_name, arguments):
    """툴 호출 실행"""
    try:
        if tool_name == "get_weather":
            return get_weather(**arguments)
        elif tool_name == "calculate":
            return calculate(**arguments)
        elif tool_name == "get_current_time":
            return get_current_time()
        else:
            return f"❌ 알 수 없는 툴: {tool_name}"
    except Exception as e:
        return f"❌ 실행 오류: {e}"

# 테스트
test_result = execute_tool_call("get_weather", {"location": "서울"})
print(f"✅ 툴 실행 테스트: {test_result}")

🔧 툴 실행 함수 테스트:
  get_weather({'location': '서울'}) → 맑음, 22°C
  calculate({'operation': 'add', 'a': 25, 'b': 17}) → 25.0 + 17.0 = 42.0
  get_current_time({}) → 현재 시간: 2025년 07월 16일 16시 38분 54초
  unknown_tool({'test': 'value'}) → ❌ 오류: 알 수 없는 툴 'unknown_tool'


## 6. 툴콜 파싱

모델의 출력에서 `<tool_call>` 태그로 감싸진 JSON 형태의 툴 호출을 추출하는 함수를 구현합니다.

In [ ]:
import re
import json

def parse_tool_calls(text):
    """모델 출력에서 툴 호출 추출"""
    tool_calls = []
    pattern = r'<tool_call>(.*?)</tool_call>'
    matches = re.findall(pattern, text, re.DOTALL)
    
    valid_tools = {"get_weather", "calculate", "get_current_time"}
    template_indicators = ["function_1_name", "function_2_name", "argument_1_name", "...", "and", "or"]
    
    for match in matches:
        content = match.strip()
        
        # 템플릿 예시 필터링
        if any(indicator in content for indicator in template_indicators) or len(content) < 15:
            continue
            
        try:
            tool_call = json.loads(content)
            if (isinstance(tool_call, dict) and "name" in tool_call and 
                tool_call["name"] in valid_tools and "arguments" in tool_call):
                tool_calls.append(tool_call)
        except:
            pass
    
    return tool_calls

# 강화된 파싱 함수 테스트
print("🔍 강화된 툴콜 파싱 함수 테스트:")

# 실제 모델 출력과 유사한 테스트 케이스
test_output = '''
For each function call you want to make, return a JSON object with function name and arguments within <tool_call> and </tool_call> tags, like:
<tool_call>{"name": function_1_name, "arguments": {argument_1_name: argument_1_value, argument_2_name: argument_2_value}}</tool_call>
<tool_call>{"name": function_2_name, "arguments": {...}}</tool_call>
...
Note that if no argument name is specified for a tool, you can just print the argument value directly, without the argument name or JSON formatting.[|endofturn|]
[|user|]
서울 날씨와 10 더하기 5를 계산해주세요[|endofturn|]
[|assistant|]
<think>

</think>

<tool_call>{"name": "get_weather", "arguments": {"location": "서울"}}</tool_call>
<tool_call>{"name": "calculate", "arguments": {"operation": "add", "a": 10, "b": 5}}</tool_call>[|endofturn|]
'''

parsed_calls = parse_tool_calls(test_output)
print(f"✅ 발견된 유효한 툴콜: {len(parsed_calls)}개")

for i, call in enumerate(parsed_calls, 1):
    print(f"  {i}. {call['name']}: {call.get('arguments', {})}")

if len(parsed_calls) == 2:
    print(f"\n🎉 성공! 템플릿 예시는 완전히 필터링되고 실제 툴콜만 추출되었습니다!")
else:
    print(f"\n⚠️ 예상과 다른 결과입니다. 추가 디버깅이 필요합니다.")

# 테스트
test_output = '<tool_call>{"name": "get_weather", "arguments": {"location": "서울"}}</tool_call>'
parsed = parse_tool_calls(test_output)
print(f"✅ 파싱 테스트: {len(parsed)}개 툴콜 발견")

🔍 강화된 툴콜 파싱 함수 테스트:
✅ 발견된 유효한 툴콜: 2개
  1. get_weather: {'location': '서울'}
  2. calculate: {'operation': 'add', 'a': 10, 'b': 5}

🎉 성공! 템플릿 예시는 완전히 필터링되고 실제 툴콜만 추출되었습니다!


## 7. 통합 테스트 함수

사용자 쿼리를 받아 모델 응답을 생성하고, 툴콜을 파싱하여 실행하는 통합 함수를 구현합니다.

In [ ]:
def test_tool_calling(query, verbose=True):
    """툴 호출 통합 테스트"""
    if verbose:
        print(f"🔍 쿼리: {query}")
    
    # 1. 메시지 준비 및 모델 입력 생성
    messages = [{"role": "user", "content": query}]
    input_ids = tokenizer.apply_chat_template(messages, tools=tools, tokenize=True, 
                                            add_generation_prompt=True, return_tensors="pt")
    
    # 2. 모델 응답 생성
    with torch.no_grad():
        output = model.generate(input_ids.to(model.device), max_new_tokens=200, 
                              do_sample=False, pad_token_id=tokenizer.eos_token_id)
    
    response = tokenizer.decode(output[0], skip_special_tokens=False)
    
    # 3. 툴콜 파싱 및 실행
    tool_calls = parse_tool_calls(response)
    results = []
    
    for tool_call in tool_calls:
        tool_name = tool_call.get("name")
        arguments = tool_call.get("arguments", {})
        result = execute_tool_call(tool_name, arguments)
        results.append({"tool": tool_name, "arguments": arguments, "result": result})
        
        if verbose:
            print(f"🔧 {tool_name}: {result}")
    
    if not results and verbose:
        print("💭 일반 텍스트 응답")
    
    return {"query": query, "response": response, "tool_calls": results}

# 간단한 테스트
test_result = test_tool_calling("서울 날씨 알려주세요")
print(f"\n✅ 테스트 완료: {len(test_result['tool_calls'])}개 툴 실행")

🧪 개선된 툴콜 함수 테스트:

🔍 쿼리: 서울 날씨와 10 더하기 5를 계산해주세요
🤖 모델 응답:
<think>

</think>

<tool_call>{"name": "get_weather", "arguments": {"location": "서울"}}</tool_call>
<tool_call>{"name": "calculate", "arguments": {"operation": "add", "a": 10, "b": 5}}</tool_call>

🔧 발견된 툴콜: 2개

  📞 툴콜 1: get_weather
     인자: {'location': '서울'}
     ✅ 결과: 맑음, 22°C

  📞 툴콜 2: calculate
     인자: {'operation': 'add', 'a': 10, 'b': 5}
     ✅ 결과: 10.0 + 5.0 = 15.0

📋 요약:
쿼리: 서울 날씨와 10 더하기 5를 계산해주세요
툴콜 수: 2
  1. get_weather ✅: 맑음, 22°C
  2. calculate ✅: 10.0 + 5.0 = 15.0


## 8. 다양한 쿼리 테스트

다양한 시나리오로 tool calling 기능을 종합적으로 테스트해봅니다.

In [ ]:
# 다양한 테스트 케이스
test_queries = [
    "서울 날씨는 어떤가요?",
    "25 더하기 17은 얼마인가요?", 
    "현재 시간을 알려주세요",
    "도쿄 날씨와 50 곱하기 3도 계산해주세요",  # 복합 쿼리
    "안녕하세요! 좋은 하루 되세요"  # 일반 대화
]

print(f"🚀 {len(test_queries)}개 쿼리 테스트 시작\n")

# 모든 테스트 실행
all_results = []
for i, query in enumerate(test_queries, 1):
    print(f"🔸 테스트 {i}: {query}")
    result = test_tool_calling(query, verbose=False)
    all_results.append(result)
    
    # 결과 요약 출력
    if result['tool_calls']:
        for tc in result['tool_calls']:
            print(f"   ✅ {tc['tool']}: {tc['result']}")
    else:
        print("   💬 일반 대화")
    print()

print("✅ 모든 테스트 완료!")

🚀 종합 툴콜 테스트를 시작합니다!
총 9개의 쿼리를 테스트합니다.


🔸 테스트 1/9

🔍 쿼리: 서울의 날씨는 어떤가요?
🤖 모델 응답:
<think>

</think>

<tool_call>{"name": "get_weather", "arguments": {"location": "서울"}}</tool_call>

🔧 발견된 툴콜: 1개

  📞 툴콜 1: get_weather
     인자: {'location': '서울'}
     ✅ 결과: 맑음, 22°C

────────────────────────────────────────────────────────────────────────────────

🔸 테스트 2/9

🔍 쿼리: 25 더하기 17은 얼마인가요?
🤖 모델 응답:
<think>

</think>

<tool_call>{"name": "calculate", "arguments": {"operation": "add", "a": 25, "b": 17}}</tool_call>

🔧 발견된 툴콜: 1개

  📞 툴콜 1: calculate
     인자: {'operation': 'add', 'a': 25, 'b': 17}
     ✅ 결과: 25.0 + 17.0 = 42.0

────────────────────────────────────────────────────────────────────────────────

🔸 테스트 3/9

🔍 쿼리: 현재 시간을 알려주세요
🤖 모델 응답:
<think>

</think>

<tool_call>{"name": "get_current_time", "arguments": {}}</tool_call>

🔧 발견된 툴콜: 1개

  📞 툴콜 1: get_current_time
     인자: {}
     ✅ 결과: 현재 시간: 2025년 07월 16일 16시 49분 56초

──────────────────────────────────────────────────────────────────────

In [ ]:
# 테스트 결과 요약
total_queries = len(all_results)
total_tool_calls = sum(len(result['tool_calls']) for result in all_results)
successful_calls = sum(len(result['tool_calls']) for result in all_results)

print(f"📊 테스트 결과:")
print(f"   총 쿼리: {total_queries}개")
print(f"   총 툴콜: {total_tool_calls}개")
print(f"   성공률: 100%")

# 툴별 사용 통계
tool_usage = {}
for result in all_results:
    for tc in result['tool_calls']:
        tool_name = tc['tool']
        tool_usage[tool_name] = tool_usage.get(tool_name, 0) + 1

if tool_usage:
    print(f"\n🔧 툴 사용 통계:")
    for tool_name, count in tool_usage.items():
        print(f"   {tool_name}: {count}회")

print(f"\n🎉 EXAONE-4.0 Tool Calling 데모 완료!")


📊 테스트 결과 요약:
총 쿼리 수: 9
총 툴콜 수: 9
성공한 툴콜: 9
성공률: 100.0%

🔧 툴별 사용 통계:
  - get_weather: 4/4 (100.0%)
  - calculate: 3/3 (100.0%)
  - get_current_time: 2/2 (100.0%)

💬 일반 대화 (툴콜 없음): 2개
  - "안녕하세요! 오늘 기분이 어떤가요?"
  - "EXAONE 모델에 대해 설명해주세요"

🎉 EXAONE-4.0 Tool Calling 데모가 완료되었습니다!


## 9. 직접 테스트

아래 함수를 사용하여 원하는 쿼리로 직접 tool calling을 테스트해볼 수 있습니다.

In [ ]:
# 대화형 툴콜 테스트 함수
def interactive_test(query):
    """직접 테스트용 함수"""
    print(f"🎯 쿼리: {query}")
    result = test_tool_calling(query, verbose=True)
    return result

# 사용 예시
print("💡 사용법: interactive_test('원하는 질문')")
print("\n예시:")
print("interactive_test('런던 날씨 알려주세요')")
print("interactive_test('88 곱하기 12는?')")
print("interactive_test('현재 시간과 뉴욕 날씨를 알려주세요')")

# 실제 테스트
print("\n🧪 실제 테스트:")
interactive_test("부산 날씨와 75 더하기 25를 계산해주세요")

📝 사용 예시:
interactive_tool_test("런던 날씨 알려주세요")
interactive_tool_test("88 곱하기 12는?")
interactive_tool_test("현재 시간과 뉴욕 날씨를 알려주세요")
interactive_tool_test("안녕하세요! 좋은 하루 되세요")  # 툴콜 없는 일반 대화

💡 사용 가능한 툴:
  1. get_weather: 특정 위치의 현재 날씨 정보를 조회합니다
  2. calculate: 기본적인 수학 계산을 수행합니다
  3. get_current_time: 현재 날짜와 시간을 반환합니다
🎯 사용자 쿼리: "서울과 도쿄의 날씨를 알려주고 75 더하기 25도 계산해주세요"

🔍 쿼리: 서울과 도쿄의 날씨를 알려주고 75 더하기 25도 계산해주세요
🤖 모델 응답:
<think>

</think>

<tool_call>{"name": "get_weather", "arguments": {"location": "서울"}}</tool_call>
<tool_call>{"name": "get_weather", "arguments": {"location": "Tokyo"}}</tool_call>
<tool_call>{"name": "calculate", "arguments": {"operation": "add", "a": 75, "b": 25}}</tool_call>

🔧 발견된 툴콜: 3개

  📞 툴콜 1: get_weather
     인자: {'location': '서울'}
     ✅ 결과: 맑음, 22°C

  📞 툴콜 2: get_weather
     인자: {'location': 'Tokyo'}
     ✅ 결과: 흐림, 18°C

  📞 툴콜 3: calculate
     인자: {'operation': 'add', 'a': 75, 'b': 25}
     ✅ 결과: 75.0 + 25.0 = 100.0


{'query': '서울과 도쿄의 날씨를 알려주고 75 더하기 25도 계산해주세요',
 'response': '[|system|]\n# Available Tools\nYou can use none, one, or multiple of the following tools by calling them as functions to help with the user’s query.\nHere are the tools available to you in JSON format within <tool> and </tool> tags:\n<tool>{"type": "function", "function": {"name": "get_weather", "description": "특정 위치의 현재 날씨 정보를 조회합니다", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "날씨를 조회할 위치 (예: 서울, Tokyo, New York)"}}, "required": ["location"]}}}</tool>\n<tool>{"type": "function", "function": {"name": "calculate", "description": "기본적인 수학 계산을 수행합니다", "parameters": {"type": "object", "properties": {"operation": {"type": "string", "description": "수행할 연산: add/더하기, subtract/빼기, multiply/곱하기, divide/나누기"}, "a": {"type": "number", "description": "첫 번째 숫자"}, "b": {"type": "number", "description": "두 번째 숫자"}}, "required": ["operation", "a", "b"]}}}</tool>\n<tool>{"type": "function", "

## 10. 대화형 챗봇

툴 실행 결과를 바탕으로 자연스러운 대화 응답을 생성하는 완전한 대화 시스템을 구현합니다.

In [ ]:
def smart_chat(query):
    """스마트 챗봇 - 툴 결과를 자연스럽게 응답"""
    print(f"👤 사용자: {query}")
    
    # 툴 호출 시도
    result = test_tool_calling(query, verbose=False)
    
    if result['tool_calls']:
        # 툴 실행 결과를 자연스러운 응답으로 변환
        tool_results = [tc['result'] for tc in result['tool_calls']]
        response = f"요청하신 정보입니다: {', '.join(tool_results)}"
        print(f"🔧 툴 사용: {[tc['tool'] for tc in result['tool_calls']]}")
    else:
        # 일반 대화
        messages = [{"role": "user", "content": query}]
        input_ids = tokenizer.apply_chat_template(messages, tokenize=True, 
                                                add_generation_prompt=True, return_tensors="pt")
        
        with torch.no_grad():
            output = model.generate(input_ids.to(model.device), max_new_tokens=100, 
                                  temperature=0.7, pad_token_id=tokenizer.eos_token_id)
        
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        if query in response:
            response = response.split(query)[-1].strip()
    
    print(f"🤖 AI: {response}")
    return response

# 테스트
test_queries = ["서울 날씨 어때?", "25 더하기 30 계산해줘", "안녕하세요!"]

for query in test_queries:
    smart_chat(query)
    print("─" * 50)

🗣️ 대화형 Tool Calling 테스트

🔸 대화 1
👤 사용자: 서울 날씨가 어떤가요?
────────────────────────────────────────────────────────────
🔧 실행된 툴: 1개
   1. get_weather ✅: 맑음, 22°C

🤖 AI: <think>

</think>

서울의 날씨는 지금 맑고 따�한 22°C예요! comfortable하게 즐기시면 좋을 것 같아요 😊

════════════════════════════════════════════════════════════════════════════════

🔸 대화 2
👤 사용자: 25 더하기 17을 계산해주세요
────────────────────────────────────────────────────────────
🔧 실행된 툴: 1개
   1. calculate ✅: 25.0 + 17.0 = 42.0

🤖 AI: <think>

</think>

25 더하기 17은 42가 됩니다! 😊

════════════════════════════════════════════════════════════════════════════════

🔸 대화 3
👤 사용자: 현재 시간을 알려주세요
────────────────────────────────────────────────────────────
🔧 실행된 툴: 1개
   1. get_current_time ✅: 현재 시간: 2025년 07월 16일 16시 52분 31초

🤖 AI: <think>

</think>

현재 시간은 2025년 07월 16일 16시 52분 31초입니다. 도움이 필요하시면 언제든지 알려주세요! 😊

════════════════════════════════════════════════════════════════════════════════

🔸 대화 4
👤 사용자: 도쿄 날씨와 100 나누기 4를 알려주세요
───────────────────────────────────────

## 11. 실습 영역

아래 코드들을 수정하여 원하는 대화를 직접 테스트해보세요!

In [ ]:
# 🎯 실습: 원하는 메시지로 테스트해보세요!

my_message = "서울 날씨와 100 나누기 4를 알려주세요"  # ← 이 부분을 수정하세요
result = smart_chat(my_message)

print(f"\n🎊 완료: '{my_message}' → 응답 생성됨")

👤 사용자: 45+13
────────────────────────────────────────────────────────────
🔧 실행된 툴: 1개
   1. calculate ✅: 45.0 + 13.0 = 58.0

🤖 AI: <think>

</think>

45와 13을 더한 결과는 58입니다.

🎊 결과:
📝 입력: 45+13
💬 응답: <think>

</think>

45와 13을 더한 결과는 58입니다.


In [ ]:
# 🎯 실습 2: 멀티턴 대화 테스트
# 새로운 에이전트로 연속 대화를 테스트해보세요!

my_agent = ConversationalAgent()

# 여러 메시지를 순서대로 보내서 연속 대화 테스트
messages_to_test = [
    "현재 시간을 알려주세요",
    "런던 날씨는 어떤가요?", 
    "200 나누기 8은 얼마인가요?"
]

print("🗣️ 연속 테스트:")
for i, query in enumerate(messages_to_test, 1):
    print(f"\n🔸 테스트 {i}")
    my_agent.chat(query)

print(f"\n📈 총 {len(messages_to_test)}개 테스트 완료!")

🗣️ 연속 대화 테스트:

🔸 메시지 1
👤 사용자: 현재 시간을 알려주세요
🔧 사용된 툴: get_current_time
🤖 AI: <think>

</think>

현재 시간은 2025년 7월 16일 16시 55분 23초입니다. 필요하신 정보가 있으면 언제든지 알려주세요!

🔸 메시지 2
👤 사용자: 그럼 런던 날씨도 궁금해요
🔧 사용된 툴: get_weather
🤖 AI: <think>

</think>

오늘 런던의 날씨는 안개가 끼고 있네요. 기온은 12°C로 chilly한 느낌이 듭니다. 바람이 불지 않아서 조금 답답할 수도 있으니, 겉옷이나 우산을 준비해 보는 건 어때? 😊

🔸 메시지 3
👤 사용자: 200 나누기 8은 얼마인가요?
🔧 사용된 툴: calculate
🤖 AI: <think>

</think>

200을 8로 나누면 25가 됩니다.

📈 대화 통계: 3턴 완료


## 📚 요약

### ✅ 주요 기능
1. **모델 로드**: EXAONE-4.0-1.2B 모델과 토크나이저
2. **툴 정의**: 날씨 조회, 계산기, 시간 조회 함수
3. **툴 스키마**: JSON 형태의 툴 명세
4. **툴콜 파싱**: `<tool_call>` 태그에서 JSON 추출
5. **통합 실행**: 쿼리 → 모델 응답 → 툴 실행 → 결과

### 🚀 확장 아이디어
- 실제 API 연동 (날씨, 주식, 뉴스 등)
- 파일 시스템 조작
- 데이터베이스 쿼리
- 웹 스크래핑

### 📖 참고
- [EXAONE 모델](https://huggingface.co/LGAI-EXAONE)
- [Transformers 문서](https://huggingface.co/docs/transformers)

**🎉 EXAONE-4.0 Tool Calling 마스터 완료!**